In [1]:
import numpy as np
from queue import PriorityQueue
from time import time
import pandas as pd
from tqdm.notebook import tqdm
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Helper functions

In [3]:
def generate_weighted_graph(nodes_no, outfile, range_from=1, range_to=10):
  graph = list(range(nodes_no))
  rd = np.random.randint(0, 2, (nodes_no, nodes_no))
  np.fill_diagonal(rd, 0)
  edges_no = int(np.sum(rd))
  with open(outfile, "w") as f:
    f.write(str(nodes_no)+" "+str(edges_no)+"\n")
    for i in range(1, nodes_no+1):
      for j in range(1, nodes_no+1):
        if rd[i-1][j-1] == 1:
          weight = np.random.randint(range_from, range_to)
          f.write(str(i)+" "+str(j)+" "+str(weight)+"\n")


In [4]:
def read_weighted_graph_from_file(textfile):
  with open(textfile) as f:
    first_line = f.readline()
  nodes, edges = first_line.split()
  nodes = int(nodes)
  edges = int(edges)
  g = np.loadtxt(textfile, skiprows=1)
  distances = np.ones((nodes, nodes))*np.Inf
  for i in range(0, nodes):
    distances[i, i] = 0
  for i in range(g.shape[0]):
    distances[int(g[i, 0])-1, int(g[i, 1])-1] = g[i, 2]
    distances[int(g[i, 1])-1, int(g[i, 0])-1] = g[i, 2]
  return nodes, edges, distances

# Dijkstra

In [5]:
def dijkstra_save(start, d, parents, outfile):
  for i in range(start+1, len(d)):
    edges = -1
    path = str()
    j = i
    while j != -1:
      edges += 1
      path = ','+str(j) +path
      j = int(parents[j])
    with open(outfile, 'a+') as f:
      line = '{} {} {} {} {}\n'.format(start, i, edges, d[i], path[1:])
      f.write(line)

In [6]:
def dijkstra(infile, outfile):
  nodes, edges, distances = read_weighted_graph_from_file(infile)
  t = time()
  for start in tqdm(range(nodes-1)):
    # start = 0
    d = np.ones((nodes))*np.Inf
    d[start] = 0
    parents = np.ones((nodes))*(-1)
    # make priority queue
    Q = PriorityQueue()
    for i in range(nodes):
      Q.put((d[i], i)) # distance from start, node number
    while not Q.empty():
      dist, node = Q.get()
      for i in range(nodes):
        w = distances[node, i]
        if i == node or np.isinf(w):
          continue
        else:
          if d[node] + w < d[i]:
            d[i] = d[node] + w
            parents[i] = node
            Q.put((d[i], i))
    dijkstra_save(start, d, parents, outfile)
  routes = sum(1 for _ in open(outfile))
  return nodes, edges, routes, time()-t

# Prim

In [7]:
def prim_save(nodes, edges, length, spanning_tree, outfile):
  with open(outfile, 'w') as f:
    f.write('{} {} {}\n'.format(nodes, edges, length))
    for node in spanning_tree:
        f.write("{} ".format(node))


def prim(infile, outfile):
  nodes, edges, distances = read_weighted_graph_from_file(infile)
  descendants = np.tile(np.arange(nodes), (nodes, 1))
  t = time()
  # Initialise
  length = 0
  spanning_tree = [0]
  spanning_process = []
  outside = [i for i in range(1, nodes)]
  while len(spanning_tree) < nodes:
    mesh = np.ix_(spanning_tree, outside)
    dists_to_check = distances[mesh]

    pos = np.where(dists_to_check == dists_to_check.min())
    spanning_process.append(spanning_tree[pos[0][0]])
    length += dists_to_check[pos[0][0], pos[1][0]]
    new_el = descendants[mesh][pos[0][0], pos[1][0]]
    spanning_process.append(new_el)
    spanning_tree.append(new_el)
    outside.remove(new_el)
  prim_save(nodes, edges, length, spanning_process, outfile)
  return nodes, length, time()-t

# Kruskal

In [8]:
def read_kruskal(textfile):
  with open(textfile) as f:
    first_line = f.readline()
  nodes, edges = first_line.split()
  nodes = int(nodes)
  edges = int(edges)
  g = np.loadtxt(textfile, skiprows=1)
  return nodes, edges, g

In [9]:
def is_not_spanning(forest, nodes):
  if len(forest) == 1:
    return False
  for tree in forest:
    if len(tree) == nodes:
      return False
  return True

In [10]:
def check_if_looping(min_edge, forest):
  for tree in forest:
    if all(i in tree for i in min_edge):
      return True
  return False

def merge_trees(min_edge, forest):
  idx_0 = -1
  idx_1 = -1
  for i in range(len(forest)):
    tree = forest[i]
    if min_edge[0] in tree:
      idx_0 = i
    elif min_edge[1] in tree:
      idx_1 = i
    if idx_0 != -1 and idx_1 != -1:
      break
  forest[idx_0].extend(forest[idx_1])
  del forest[idx_1]
  return forest

In [11]:
def kruskal_save(nodes, edges, length, spanning_tree, outfile):
  with open(outfile, 'w') as f:
    f.write('{} {} {}\n'.format(nodes, edges, length))
    for node in spanning_tree:
        f.write("{} ".format(node))
  

In [12]:
def kruskal(infile, outfile):
  nodes, edges, g = read_kruskal(infile)
  g = g[g[:, 2].argsort()]
  t = time()
  length = 0
  path = []
  forest = [[i] for i in range(nodes)]
  while g.shape[0] != 0 and is_not_spanning(forest, nodes):
    min_dist = g[0, 2]
    min_edge = g[0, :-1]
    if not check_if_looping(min_edge, forest):
      forest = merge_trees(min_edge, forest)
      length += min_dist
      path.append(min_edge[0])
      path.append(min_edge[1])
    g = g[1:, ]
  
  kruskal_save(nodes, edges, length, path, outfile)
  
  return nodes, length, time()-t

# Testing

In [13]:
infiles = ["g7.txt", "g9.txt", "g10.txt", "g100.txt", "g200.txt", "g500.txt"]


## Dijkstra

In [14]:
measures_dijkstra = ["Input filename", "Output filename", 
                     "Nodes", "Edges", "Routes", "Times"]
outfiles_dijkstra = []
n = []
e = []
r = []
times = []

for i in tqdm(range(len(infiles))):
  outfile = infiles[i][:-4]+"oD.txt"

  try: # to not append to solution from previous session
    os.remove("/content/drive/My Drive/AaDS/greedy_algorithms/"+outfile)
  except OSError:
    pass

  nodes, edges, routes, t = dijkstra("/content/drive/My Drive/AaDS/greedy_algorithms/"+infiles[i],
                                     "/content/drive/My Drive/AaDS/greedy_algorithms/"+outfile)
  outfiles_dijkstra.append(outfile)
  n.append(nodes)
  e.append(edges)
  r.append(routes)
  times.append(t)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/199 [00:00<?, ?it/s]

  0%|          | 0/499 [00:00<?, ?it/s]

In [15]:
df = pd.DataFrame(list(zip(infiles, outfiles_dijkstra, n, e, r, times)),
               columns=measures_dijkstra)
df

,Input filename,Output filename,Nodes,Edges,Routes,Times
0,g7.txt,g7oD.txt,7,11,21,0.334550
1,g9.txt,g9oD.txt,9,14,36,0.393891
2,g10.txt,g10oD.txt,10,39,45,0.460295
3,g100.txt,g100oD.txt,100,4931,4950,29.513317
4,g200.txt,g200oD.txt,200,19909,19900,131.685232
5,g500.txt,g500oD.txt,500,125187,124750,1401.342465


## Minimum spanning tree

In [16]:
measures_msp = ["Input", "Out P", "Out K", 
                "Nodes", "Length P", "Length K",
                "Times P", "Times K"]

outfiles_p = []
outfiles_k = []

n = []

l_p = []
l_k = []

times_p = []
times_k = []

for i in tqdm(range(len(infiles))):
  # Prim
  outfile = infiles[i][:-4]+"oP.txt"
  nodes, l, t = prim("/content/drive/My Drive/AaDS/greedy_algorithms/"+infiles[i],
                     "/content/drive/My Drive/AaDS/greedy_algorithms/"+outfile)
  outfiles_p.append(outfile)
  n.append(nodes)
  l_p.append(l)
  times_p.append(t)
  # Kruskal
  outfile = infiles[i][:-4]+"oK.txt"
  _, l, t = kruskal("/content/drive/My Drive/AaDS/greedy_algorithms/"+infiles[i],
                    "/content/drive/My Drive/AaDS/greedy_algorithms/"+outfile)
  outfiles_k.append(outfile)
  l_k.append(l)
  times_k.append(t)

  0%|          | 0/6 [00:00<?, ?it/s]

In [17]:
df = pd.DataFrame(list(zip(infiles, outfiles_p, outfiles_k,
                           n, l_p, l_k,
                           times_p, times_k)),
               columns=measures_msp)
df

,Input,Out P,Out K,Nodes,Length P,Length K,Times P,Times K
0,g7.txt,g7oP.txt,g7oK.txt,7,24.0,24.0,0.009195,0.006592
1,g9.txt,g9oP.txt,g9oK.txt,9,37.0,37.0,0.254509,0.182723
2,g10.txt,g10oP.txt,g10oK.txt,10,10.0,9.0,0.180625,0.182348
3,g100.txt,g100oP.txt,g100oK.txt,100,99.0,99.0,0.203712,0.214441
4,g200.txt,g200oP.txt,g200oK.txt,200,199.0,199.0,0.232922,0.297085
5,g500.txt,g500oP.txt,g500oK.txt,500,499.0,499.0,0.632487,0.794771
